In [17]:
import pandas as pd
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline
import sentencepiece
from germansentiment import SentimentModel

In [5]:
df_german = pd.read_csv('../location_analysis/data/df_german_with_mentions.csv')

In [8]:
german_sentiment_model = SentimentModel()


In [9]:
def analyze_german_sentiment(title):
    classes, probabilities = german_sentiment_model.predict_sentiment([title], output_probabilities=True)
    prob_dict = {sentiment: prob for sentiment, prob in probabilities[0]}
    prob_pos = round(prob_dict.get('positive', 0), 5)
    prob_neg = round(prob_dict.get('negative', 0), 5)
    prob_neu = round(prob_dict.get('neutral', 0), 5)
    return classes[0], prob_pos, prob_neg, prob_neu


In [10]:
df_german[['Sentiment', 'Probability_Pos', 'Probability_Neg', 'Probability_Neu']] = df_german['Title'].apply(lambda title: pd.Series(analyze_german_sentiment(title)))


In [20]:
def set_score(row):
    if row['Sentiment'] == 'positive':
        return row['Probability_Pos']
    elif row['Sentiment'] == 'negative':
        return row['Probability_Neg']
    elif row['Sentiment'] == 'neutral':
        return row['Probability_Neu']
    return None

In [23]:
df_german['Confidence'] = df_german.apply(set_score, axis=1)
df_german = df_german.reindex(columns=['Title', 'Header', 'Content', 'Mentioned_Countries', 'Mentioned_Swiss_Cities', 'Sentiment', 'Confidence', 'Probability_Neu','Probability_Neg', 'Probability_Pos'])

Export

In [29]:
file_path = '../location_analysis/data/df_german_with_mentions_sentiment.csv'
df_german.to_csv(file_path)